#                                Data Mining : Assignment 3

##                                          Naive Bayes Classifier

###                                                      Sameer Valiah Pusunuru
###                                                              1001552473

In [236]:

import os
import re
import tarfile
import nltk
import operator
import numpy as np
from six.moves import urllib
from nltk.corpus import stopwords 


geting file from the url path and then downloading it ,verfiying it and storing it

In [237]:
DOWNLOADED_FILENAME = 'data.tar.gz'

def download_file(url_path):
    if not os.path.exists(DOWNLOADED_FILENAME):
        filename, _ = urllib.request.urlretrieve(url_path, DOWNLOADED_FILENAME)

    print('Found and verified file from this path: ', url_path)
    print('Downloaded file: ', DOWNLOADED_FILENAME)

after getting folder with data going into each folder and then storing data bases on postive negative reviews of train and test set also to remove the any charcater which is not alphabet or number 

In [238]:
#regular expression to remove charcters other then alphabets or numbers
TOKEN_REGEX = re.compile("[^A-Za-z0-9 ]+")
#function to clean each review
def get_reviews(dirname, positive=True):
    label = 1 if positive else 0

    reviews = []
    for filename in os.listdir(dirname):
        if filename.endswith(".txt"):
            with open(dirname + filename, 'r+', encoding="utf8") as f:
                review = f.read()
                review = review.lower().replace("<br />", " ")
                review = re.sub(TOKEN_REGEX, '', review)
                reviews.append((review, label))
    
    return reviews 
#function to take one input at time and the clean the input using another function
def extract_reviews():
    if not os.path.exists('aclImdb'):
        with tarfile.open(DOWNLOADED_FILENAME) as tar:
            tar.extractall()
            tar.close()   
    positive_reviews = get_reviews("aclImdb/train/pos/", positive=True)
    positive_reviews_test = (get_reviews("aclImdb/test/pos/", positive=True))
    negative_reviews = get_reviews("aclImdb/train/neg/", positive=False)
    negative_reviews_test = get_reviews("aclImdb/test/neg/", positive=False)
    return positive_reviews, negative_reviews, positive_reviews_test, negative_reviews_test

In [37]:
#web url for the folder
URL_PATH = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
download_file(URL_PATH)

Found and verified file from this path:  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Downloaded file:  ImdbReviews.tar.gz


In [239]:
#stroing all the reviews 
positive_reviews, negative_reviews,positive_reviews_test,negative_reviews_test = extract_reviews()

In [240]:

total_positive_reviews = positive_reviews_test+positive_reviews

len(total_positive_reviews)


25000

In [241]:
total_negative_reviews =negative_reviews+negative_reviews_test

len(total_negative_reviews)

25000

Diving the whole data in three parts DEV, TRAIN , TEst whith DEV and Train total to be 70% ~ 35000

In [288]:
TRAIN_DATA = 17500
train_reviews_positive = total_positive_reviews[:TRAIN_DATA] 
train_reviews_negative = total_negative_reviews[:TRAIN_DATA]
test_reviews_postive = total_positive_reviews[TRAIN_DATA:]
test_reviews_negative = total_negative_reviews[TRAIN_DATA:]
total_test = test_reviews_postive+test_reviews_negative



downloading stop woords and storing them so that we can remove them from our word count

In [243]:


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


funtion to count number of words in the review and there number of accourances also removing the words which are repeated less then 5 times

In [276]:
def get_vocabulary(train_reviews):
    words_set = {}
    for review in train_reviews:
        for r in review[0].split():
            if r in stop_words:
                continue
            else:
                words_set[r] = words_set.get(r, 0) + 1
    #removing word which are repeated less then 5 times
    words_set_copy = {}           
    for word in words_set:
        val = words_set.get(word)
        if(val>5):
            words_set_copy[word] = val;
    return words_set_copy

As we have to use k fold which is mentiones as 5 in the requirement ,I am diving my train and dev data into 5 parts with equal postive and negative reviews

In [277]:

#k fold :where k is 5 hence 5 sets of train data

#set 1
set1 = train_reviews_negative[:3500]+train_reviews_positive[:3500]
set1train = train_reviews_negative[3500:]+ train_reviews_positive[3500:]

#set2
set2 = train_reviews_negative[3500:7000]+train_reviews_positive[3500:7000]
set2train = train_reviews_negative[:3500]+train_reviews_negative[7000:]+ train_reviews_positive[:3500]+train_reviews_positive[7000:]

#set3
set3 = train_reviews_negative[7000:10500]+train_reviews_positive[7000:10500]
set3train = train_reviews_negative[:7000]+train_reviews_negative[10500:]+ train_reviews_positive[:7000]+train_reviews_positive[10500:]

#set4
set4 = train_reviews_negative[10500:14000]+train_reviews_positive[10500:14000]
set4train = train_reviews_negative[:10500]+train_reviews_negative[14000:]+ train_reviews_positive[:10500]+train_reviews_positive[14000:]

#set5
set5 = train_reviews_negative[14000:]+train_reviews_positive[14000:]
set5train = train_reviews_negative[:14000]+train_reviews_positive[:14000]



function to find totalnumber of word in postive and negative reviews vocabulary without repetiton 
function to find the total number of postive/negative words count


In [323]:
def total(negative,positive):
    words_total = set()
    
    for review in negative:
        for r in review[0].split():
            words_total.add(r)
    for review in positive:
        for r in review[0].split():
            words_total.add(r)
        
        
    return(len(words_total))

def totalWords(words):
    count =0
    for p in words:
        count = count + words.get(p);
        
    return count
def top10Words():
    train_vocabulary_positive =get_vocabulary(train_reviews_positive)
    train_vocabulary_negative =get_vocabulary(train_reviews_negative)
    sorted_train_vocabulary_negative = sorted(train_vocabulary_negative.items(), key=operator.itemgetter(1),reverse = True)
    sorted_train_vocabulary_positive = sorted(train_vocabulary_positive.items(), key=operator.itemgetter(1),reverse = True)
    return sorted_train_vocabulary_negative,sorted_train_vocabulary_positive


functions for finding probabilty using smoothing and without smoothing

In [279]:

#the common probabilty finding by counts of words[i] in postive/negative divided by total postive/negative words count
#where review is array of words
def prob(setl,negativelen,positivelen,set1_vocabulary_negative,set1_vocabulary_positive):
    count=0; 
    for review in setl:
        val=0
        for r in review[0].split():
            pos=1
            neg=1
            if r in stop_words:
                continue
            else:
                posval = set1_vocabulary_positive.get(r,0);
                negval = set1_vocabulary_negative.get(r,0);
                pos = (posval/positivelen)*pos
                neg = (negval/negativelen)*neg
        
        if(pos>neg):
            val=1;
        if(val== review[1])  :  
            count= count+1;
    
    return (count/len(setl))*100;
    

The major problem with the basic probabilyt finding will be when a  word in review will not be found in postive vocabulary or negative vocabulary we get 0 which will make the whole probabilty 0, hence we require smoothing to make more effecient predications I am using Laplace smoothing

In [314]:
#in this funtion we techincally do the same as the above but we add 1 to both 
def probSmooth(setl,wordlen,negativelen,positivelen,set1_vocabulary_negative,set1_vocabulary_positive):
    count=0; 
    for review in setl:
        val=0
        pos=1
        neg=1
        for r in review[0].split():
            
            if r in stop_words:
                continue
            else:
                posval = set1_vocabulary_positive.get(r,0)+1;
                negval = set1_vocabulary_negative.get(r,0)+1;
                pos = (posval/(positivelen+wordlen)+1)*pos
                neg = (negval/(negativelen+wordlen)+1)*neg
        
        if(pos>neg):
            val=1;
        if(val== review[1])  :  
            count= count+1;
    
    return (count/len(setl))*100;

In [315]:
#1 fold 
set1_vocabulary_negative =get_vocabulary(set1train[:14000])
set1_vocabulary_positive =get_vocabulary(set1train[14000:])
negativelen =totalWords(set1_vocabulary_negative)
positivelen =totalWords(set1_vocabulary_positive)
wordslen = total(set1train[:14000],set1train[14000:])


accuracy1 = prob(set1,negativelen,positivelen,set1_vocabulary_negative,set1_vocabulary_positive) 
accuracy1_smooth = probSmooth(set1,wordslen,negativelen,positivelen,set1_vocabulary_negative,set1_vocabulary_positive) 
print("Accuract of 1st fold without smoothing "+str(accuracy1))
print("Accuract of 1st fold with smoothing "+str(accuracy1_smooth))

Accuract of 1st fold without smoothing 60.042857142857144
Accuract of 1st fold with smoothing 66.0


In [316]:
#2 fold 
set2_vocabulary_negative =get_vocabulary(set2train[:14000])
set2_vocabulary_positive =get_vocabulary(set2train[14000:])
negativelen =totalWords(set2_vocabulary_negative)
positivelen =totalWords(set2_vocabulary_positive)
wordslen = total(set2train[:14000],set2train[14000:])
accuracy2 = prob(set2,negativelen,positivelen,set2_vocabulary_negative,set2_vocabulary_positive) 
accuracy2_smooth = probSmooth(set2,wordslen,negativelen,positivelen,set2_vocabulary_negative,set2_vocabulary_positive)
print("Accuract of 2nd fold without smoothing "+str(accuracy2))
print("Accuract of 2nd fold with smoothing "+str(accuracy2_smooth))

Accuract of 2nd fold without smoothing 58.72857142857143
Accuract of 2nd fold with smoothing 66.52857142857142


In [317]:
#3 fold 
set3_vocabulary_negative =get_vocabulary(set3train[:14000])
set3_vocabulary_positive =get_vocabulary(set3train[14000:])
negativelen =totalWords(set3_vocabulary_negative)
positivelen =totalWords(set3_vocabulary_positive)
wordslen = total(set3train[:14000],set3train[14000:])
accuracy3 = prob(set3,negativelen,positivelen,set3_vocabulary_negative,set3_vocabulary_positive) 
accuracy3_smooth = probSmooth(set3,wordslen,negativelen,positivelen,set3_vocabulary_negative,set3_vocabulary_positive)
print("Accuract of 3rd fold without smoothing "+str(accuracy3))
print("Accuract of 3rd fold with smoothing "+str(accuracy3_smooth))

Accuract of 3rd fold without smoothing 60.08571428571429
Accuract of 3rd fold with smoothing 64.97142857142858


In [318]:
#4 fold 
set4_vocabulary_negative =get_vocabulary(set4train[:14000])
set4_vocabulary_positive =get_vocabulary(set4train[14000:])
negativelen =totalWords(set4_vocabulary_negative)
positivelen =totalWords(set4_vocabulary_positive)
wordslen = total(set4train[:14000],set4train[14000:])
accuracy4 = prob(set4,negativelen,positivelen,set4_vocabulary_negative,set4_vocabulary_positive)
accuracy4_smooth = probSmooth(set4,wordslen,negativelen,positivelen,set4_vocabulary_negative,set4_vocabulary_positive)
print("Accuract of 4th fold without smoothing "+str(accuracy4))
print("Accuract of 4th fold with smoothing "+str(accuracy4_smooth))

Accuract of 4th fold without smoothing 59.71428571428572
Accuract of 4th fold with smoothing 65.8


In [319]:
#5 fold 
set5_vocabulary_negative =get_vocabulary(set5train[:14000])
set5_vocabulary_positive =get_vocabulary(set5train[14000:])
negativelen =totalWords(set5_vocabulary_negative)
positivelen =totalWords(set5_vocabulary_positive)
wordslen = total(set5train[:14000],set5train[14000:])
accuracy5 = prob(set5,negativelen,positivelen,set5_vocabulary_negative,set5_vocabulary_positive) 
accuracy5_smooth = probSmooth(set5,wordslen,negativelen,positivelen,set5_vocabulary_negative,set5_vocabulary_positive)
print("Accuract of 5th fold without smoothing "+str(accuracy5))
print("Accuract of 5th fold with smoothing "+str(accuracy5_smooth))

Accuract of 5th fold without smoothing 60.52857142857143
Accuract of 5th fold with smoothing 66.62857142857142


In [321]:
#Average of k fold comparing the smoothing and non smoothing 
accuracy_without_smooth = (accuracy1+accuracy2+accuracy3+accuracy3+accuracy5)/5;
accuracy_smooth = (accuracy1_smooth+accuracy2_smooth+accuracy3_smooth+accuracy3_smooth+accuracy5_smooth)/5;
print("Average accuracy without smoothing:-  "+str(accuracy_without_smooth))
print("Average accuracy with smoothing:- "+str(accuracy_smooth))

Average accuracy without smoothing:-  59.894285714285715
Average accuracy with smoothing:- 65.82000000000001


Top 10 word that predict positive and negative classes :

In [341]:
positive, negative =top10Words()
print("Top 10 in positive class \n")
for i in positive[:10]:
    print(i[0])
print("\nTop 10 in negative class")
for i in negative[:10]:
    print(i[0])
    

Top 10 in positive class 

movie
film
one
like
even
bad
good
would
really
time

Top 10 in negative class
film
movie
one
like
good
great
story
see
time
well


As we have found out that using smoothing gives us better prediction we use smoothing on out test set

In [322]:
train_vocabulary_negative =get_vocabulary(train_reviews_negative)
train_vocabulary_positive =get_vocabulary(train_reviews_positive)
negativelen =totalNegativeWords(train_vocabulary_negative)
positivelen =totalPostiveWords(train_vocabulary_positive)
wordslen = total(train_reviews_negative,train_reviews_positive)
accuracy_test = probSmooth(total_test,wordslen,negativelen,positivelen,train_vocabulary_negative,train_vocabulary_positive)
print("Final accuracy :- "+ str(accuracy_test))


Final accuracy :- 65.44666666666666


References :
    
https://stackoverflow.com/questions/7971618/python-return-first-n-keyvalue-pairs-from-dict
https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
https://thispointer.com/python-how-to-add-append-key-value-pairs-in-dictionary-using-dict-update/
https://towardsdatascience.com/unfolding-na%C3%AFve-bayes-from-scratch-2e86dcae4b01